## **Ipthon Script to calculate NACorrection and Fractional Enrichment for a demo dataset.**

Prerequisite Knowledge:

**Natural Abundance Correction**- Natural abundance (NA) refers to the abundance of isotopes of a chemical element as naturally found on the planet. While performing analysis,the observed intensity contains contribution from isotopic natural abundance that needs to be corrected. This process is referred as NA Correction.

**Pool Total**- Sum of the intensities of all different number of labeled atoms of the isotope element is called pool total.

**Fractional enrichment**- Normalization of intensities of a metabolite between the range of 0 to 1.

**Welcome to the interactive Polly IPython Notebook.**

With this interactive Polly notebook you would be able to calculate NA Corrected intensities as well as fractional enrichment for LCMS/MS input file. Information on some functions used:

 - corna- package which looks into NA Correction.
 - msms.csv - demo raw_intensity file.
 - multiquant_parser.merge_mq_metadata - merge multiquant files and metadata files
 - multiquant_parser.add_mass_and_no_of_atoms_info_frm_label - from label column add information of molecular mass,    isotopic mass, total number of atoms, number of labeled atoms for parent as well as isotope fragment.
 - fractional_enrichment - Calculates fractional enrichment for the NA Corrected dataframe.

In [7]:
import pandas as pd
import numpy as np
import re

import corna.constants as const
from corna.helpers import get_isotope_na, replace_negatives_in_column
from corna.inputs.column_conventions import multiquant 
from corna.inputs import multiquant_parser
from corna.postprocess import fractional_enrichment
from corna.algorithms.background_correction import background_correction

**Defining the input files path and Natural Abundance values of elements.**

In [8]:
raw_df= pd.read_csv('raw_intensity_file.csv')
metadata_df= pd.read_excel('metadata.xlsx')
sample_metadata = pd.read_excel('metadata_sample.xlsx')
isBackground= False
isotope_dict= const.ISOTOPE_NA_MASS
REQUIRED_COL= [multiquant.FORMULA, multiquant.LABEL, multiquant.NAME, multiquant.SAMPLE, multiquant.COHORT,
                     multiquant.MQ_FRAGMENT, multiquant.INTENSITY, multiquant.PARENT_FORM,const.NA_CORRECTED_COL, 
                        const.BACKGROUND_WITH_ZERO, const.BACKGROUND_CORRECTED]

**Merge the raw_intensity dataframe with the metadata and sample metadata(if background correction to be performed).**

In [9]:
msms_df, list_of_replicates = multiquant_parser.merge_mq_metadata(raw_df, metadata_df, sample_metadata)

/home/priyanka/corna_package/env_corna/local/lib/python2.7/site-packages/corna-0.0.328-py2.7.egg/corna/inputs/multiquant_parser.py:187: UserWarning: Std samples not found inCohort Name column
  warnings.warn('Std samples not found in' + multiquant.COHORT + ' column')


In [10]:
final_df = background_correction(msms_df, list_of_replicates, isotope_dict=const.ISOTOPE_NA_MASS)
final_df = replace_negatives_in_column(final_df,const.BACKGROUND_WITH_ZERO, const.BACKGROUND_CORRECTED)
print final_df

/home/priyanka/corna_package/env_corna/local/lib/python2.7/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/priyanka/corna_package/env_corna/local/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


                              Sample                  Cohort Name  \
0     Filename_ABC.wiff (sample 426)    A. [13C-glc] Cohort1 0min   
48    Filename_ABC.wiff (sample 427)    B. [13C-glc] Cohort1 5min   
96    Filename_ABC.wiff (sample 428)   C. [13C-glc] Cohort1 15min   
144   Filename_ABC.wiff (sample 429)   D. [13C-glc] Cohort1 30min   
192   Filename_ABC.wiff (sample 430)   E. [13C-glc] Cohort1 60min   
240   Filename_ABC.wiff (sample 431)  F. [13C-glc] Cohort1 120min   
288   Filename_ABC.wiff (sample 432)  G. [13C-glc] Cohort1 240min   
336   Filename_ABC.wiff (sample 434)    I. [13C-glc] Cohort2 0min   
384   Filename_ABC.wiff (sample 435)    J. [13C-glc] Cohort2 5min   
432   Filename_ABC.wiff (sample 436)   K. [13C-glc] Cohort2 15min   
480   Filename_ABC.wiff (sample 437)   L. [13C-glc] Cohort2 30min   
528   Filename_ABC.wiff (sample 438)   M. [13C-glc] Cohort2 60min   
576   Filename_ABC.wiff (sample 439)  N. [13C-glc] Cohort2 120min   
624   Filename_ABC.wiff (sample 44

**From Label column add information of molecular mass, isotopic mass, total number of atoms, number of labeled atoms for parent as well as isotope fragment.**

In [11]:
isotracer = msms_df[multiquant.ISOTRACER].unique()
intensity_col= const.BACKGROUND_WITH_ZERO

**Get Natural abundance value of the isotracer present in the compound(na).**

In [12]:
final_df[const.NA_CORRECTED_COL]=0.0
output_df= pd.DataFrame()
metab_dict={}

na= get_isotope_na(isotracer[0], isotope_dict)

**PARENT_NUM_ ATOMS - Total number of atoms of the isotracer element in parent formula.**

**DAUGHTER_NUM_ATOMS - Total number of atoms of the isotracer element in daughter formula.**

**PARENT_NUM_LABELED_ATOMS - number of labeled atoms ofisotracer element in parent formula.**

**DAUGHTER_NUM_LABELED_ATOMS - number of labeled atoms ofisotracer element in daughter formula.**

In [13]:
final_df['A']=(1 + na * (final_df[const.PARENT_NUM_ATOMS]-final_df[const.PARENT_NUM_LABELED_ATOMS]))
final_df['B']= na * ((final_df[const.PARENT_NUM_ATOMS]-final_df[const.DAUGHTER_NUM_ATOMS]) -\
                         (final_df[const.PARENT_NUM_LABELED_ATOMS]-final_df[const.DAUGHTER_NUM_LABELED_ATOMS]-1))
final_df['C']=  na * (final_df[const.DAUGHTER_NUM_ATOMS]-final_df[const.DAUGHTER_NUM_LABELED_ATOMS]+1)

**Drop columns not required for processing.**

In [ ]:
final_df.drop([const.PARENT_MASS_MOL, const.DAUGHTER_MASS_MOL, const.PARENT_NUM_ATOMS,
                const.DAUGHTER_NUM_ATOMS, const.DAUGHTER_NUM_LABELED_ATOMS, const.PARENT_NUM_LABELED_ATOMS], axis=1, inplace=True)

**Create metabolite : intensity dictionary of the form:**

        {'SAMPLE 2_10':{
            (191, 111): 2345.75, (192, 111):5644.847
            }
        }

In [ ]:
for samp in final_df.Sample.unique():
    
    metab_df = final_df[final_df[multiquant.SAMPLE]==samp]
    frag_dict={}
    for index, row in metab_df.iterrows():
        frag_dict[(row[const.PARENT_MASS_ISO],row[const.DAUGHTER_MASS_ISO])]=row[intensity_col]
    
    metab_dict[samp]= frag_dict

**In each sample correct the intensities of daughter fragment one by one using the intensity of M+0 isotopolgue.**

In [ ]:
for samp in final_df.Sample.unique():
    metab_df = final_df[final_df[multiquant.SAMPLE]==samp]
    frag= metab_dict[samp]  
     
    for index, row in metab_df.iterrows():
        m_n= row[const.DAUGHTER_MASS_ISO]
        m_1_n= row[const.PARENT_MASS_ISO]-1
        m_n_1= row[const.DAUGHTER_MASS_ISO]-1
        intensity_m_n= row[intensity_col]
        try:
            intensity_m_1_n= frag[m_1_n, m_n]
        except KeyError:
            intensity_m_1_n=0
        try:
            intensity_m_1_n_1= frag[m_1_n, m_n_1]
        except KeyError:
            intensity_m_1_n_1= 0
        
        corrected= intensity_m_n * row['A']  - intensity_m_1_n * row['B'] -\
                                                intensity_m_1_n_1 * row['C']
        metab_df.set_value(index=index, col=const.NA_CORRECTED_COL, value= corrected)

    output_df=output_df.append(metab_df) 

**Filter the output dataframe to extract the required columns.**

In [ ]:
output_df= output_df.filter(REQUIRED_COL)
output_df = replace_negatives_in_column(output_df,const.NA_CORRECTED_WITH_ZERO, const.NA_CORRECTED_COL)
print output_df

**Calculate Fractional Enrichment**

In [ ]:
fractional_enriched_df = fractional_enrichment(output_df)
print fractional_enriched_df

In [ ]:
df= pd.merge(output_df, fractional_enriched_df, on=['Label', 'Sample', 'Name', 'Formula'])
print df